In [8]:
from sklearn.datasets import fetch_openml
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import DBSCAN

# 2

In [3]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
mnist.target = mnist.target.astype(np.uint8)
X = mnist["data"]
y = mnist["target"]

In [7]:
# filter(lambda(x: x!0, distance))

# 3.1-4

In [12]:
silhouette = []
cm = 0
for i in range(8, 13):
    kmeans = KMeans(n_clusters=i, n_init=10)
    kmeans.fit(X)
    y_pred = kmeans.predict(X)
    
    # score = silhouette_score(X, y_pred) 
    score = silhouette_score(X, kmeans.labels_)
    silhouette.append(score)
    # print(f"k = {i} silhouette score = {silhouette[-1]}")

    if i == 10:
        cm = confusion_matrix(y, y_pred)
        # print(f"k = {i} confusion matrix = \n{cm}", cm)
        
with open("kmeans_sil.pkl", "wb") as file:
    pickle.dump(silhouette, file)
print(silhouette)
print(cm)

[0.07338201924675067, 0.0568179310255884, 0.05871423980472426, 0.05834348905464586, 0.05817341754740102]
[[   2   42  173 5046    9    4 1251   80  289    7]
 [4293    6    7    0   10 3527    7    9    8   10]
 [ 427  215  151   56 4864  436  243  200  319   79]
 [ 456  191   33   22  210   53  492 1060 4578   46]
 [ 181 3735  169    9   29  229  268   16    0 2188]
 [ 165  424   70   60    7  262 1844 1161 2109  211]
 [ 200   68 4432   77   56   43 1942   16   38    4]
 [ 377 2086    4   21   53  310   13   20    5 4404]
 [ 338  208   54   37   53  316  353 4108 1171  187]
 [ 267 3456   16   50   20   92   30   89   86 2852]]


# 3.5

In [13]:
max_indices = np.argmax(cm, axis=1)
max_indices_sorted = sorted(set(max_indices))

with open("kmeans_argmax.pkl", "wb") as file:
    pickle.dump(max_indices_sorted, file)
print(f"max_indices_sorted = {max_indices_sorted}")

max_indices_sorted = [0, 1, 2, 3, 4, 7, 8, 9]


# 3.6

In [6]:
distances = np.array([np.linalg.norm(X[i] - X[j]) for i in range(300) for j in range(len(X))])
distances = [i for i in distances if i != 0]
distances = np.sort(distances)[:10]

with open("dist.pkl", "wb") as f:
    pickle.dump(list(distances), f)

print(f"distances = {distances}")

distances = [279.26152617 304.3764117  317.5893575  328.76587414 333.45464459
 352.89800226 355.17742045 358.0740147  359.64287842 360.42474943]


# 3.7-8

In [9]:
s = np.mean(distances[:3])
print(s)
eps_min = s
eps_max = s + 0.1*s
eps_step = 0.04*s
eps_list = np.arange(eps_min, eps_max, eps_step)
print(eps_list)

for eps in eps_list:
    dbscan = DBSCAN(eps=eps)
    dbscan.fit(X)
    labels = dbscan.labels_

300.40909845916684
[300.40909846 312.4254624  324.44182634]


In [10]:
label_list = []

for eps in eps_list:
    dbscan = DBSCAN(eps=eps)
    dbscan.fit(X)
    n_labels = len(np.unique(dbscan.labels_))
    label_list.append(n_labels)

with open("dbscan_len.pkl", "wb") as f:
    pickle.dump(label_list, f)
print(label_list)

[4, 7, 22]
